In [1]:
!nvidia-smi

Mon Nov 20 21:39:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install transformers==4.33.2
!pip3 install optimum==1.13.2
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
model_name_or_path = 'TheBloke/Llama-2-13B-German-Assistant-v4-GPTQ'  # "TheBloke/Llama-2-13B-chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [5]:
import pickle

with open("definitions.pickle", "rb") as file:
    definitions = pickle.load(file)

definitions

['Die "Negation:NEG_XgeschweigedennY-Konstruktion" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) "geschweige_denn", die internen Kern-Konstruktionselemente (Kern-KE) "Erstes_Konjunkt" und "Zweites_Konjunkt" sowie die externen Kern-KE "Negator" und "Fokuskontext". Das "Erste_Konjunkt" geht dem "KEE" voraus, auf welches da

In [10]:
import re
from tqdm.notebook import tqdm

def generate_examples(definition: str, temperature=0.75, top_p=0.95, top_k=1, max_new_tokens=1024):
    prompt = lambda definition: f'''Nenne fünf konkrete Beispiele für eine vordefinierte Satzkonstruktion. Der User definiert die Konstruktion. Alle Sätze müssen zur definierten Konstruktion passen. Gib nur die Sätze in einer Python-Liste aus.
    ### User: Dies ist die Definition:
    {definition}

    ### Assistant: Ausgabe:
    '''
    #prompt = lambda definition: f'''[INST] <<SYS>>
    #Tell me five example sentences for a predefined German construction.
    #The definition will be provided in German.
    #Only output the example sentence.
    #Output the example sentences in a Python list.
    #Here are the construction definitions in German:
    #{definitions}
    #<</SYS>>
    #Output: ['"""

    prompt_length = len(prompt(definition))
    output = []
    first_output = ""
    while "." not in first_output or len(output) < 5:
        input_ids = tokenizer(prompt(definition), return_tensors='pt').input_ids.cuda()
        output = model.generate(inputs=input_ids, temperature=temperature,
                                do_sample=True, top_p=top_p, top_k=top_k,
                                max_new_tokens=max_new_tokens)
        output = tokenizer.decode(output[0])[prompt_length:].strip()
        output = re.findall('\[.*\]', output)
        if len(output) > 0:
            first_output = output[0]
            # print(type(first_output))
        #print(".", end="")
        print(f"\t{output}")
    #print(f"\n{output}")
    return output

examples = []
for definition in tqdm(definitions):
    example = generate_examples(definition, temperature=1, max_new_tokens=300,
                                top_k=100, top_p=0.90)
    #example = re.findall('\[("[^"]*"(?:, ?"[^"]*")*)\]', example)
    try:
        print(">", example)
        examples.append(example)  # one is enough
    except:
        print(">", [])
        examples.append([])

with open("examples.pickle", "wb") as file:
    pickle.dump(examples, file)

  0%|          | 0/211 [00:00<?, ?it/s]

	[]
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	["['Kommt auf', 'hast du', 'dich', 'nun', 'jemals', 'vorgemerkt?', 'hast du', 'dich', 'nun', 'jemals', 'bedient?', 'dich', 'nun', 'jemals', 'gesehen?']"]
	['["Bist du auf ein Computer gesehen?", "Hast du schon einen Computer bedient?"]']
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	[]
	["['Negation:NEG_XgeschweigedennY-Konstruktion', 'ersteres_konjunkt', 'zweites_konjunkt', 'negator', 'fokuskontext']"]
	[]
	[]
	[]
	["['NEG_XgeschweigedennY', 'Negation:NEG_XgeschweigedennY-Konstruktion', 'Erstes_Konjunkt', 'Zweites_Konjunkt', 'Fokuskontext']"]
	[]
	[]
	[]
	[]
	['[1. "Die meisten von ihnen haben zuvor noch nie einen Computer gesehen", 2. "Die meisten von ihnen haben zuvor noch nie einen Computer bedient", 3. "Negation:NEG_XgeschweigedennY-Konstruktion", 4. "Negator", 5. "Erstes\\_Konjunkt", 6. "Zweites\\_Konjunkt", 7. "Fokuskontext"]']
	[]
	['["Negation:NEG_XgeschweigedennY-Konstruktion"]']
	['[["Deutsche Zungen waren früher einmal angeblich die besten in der

KeyboardInterrupt: ignored